In [7]:
!pip -q install langchain
!pip -q install guardrails-ai


[notice] A new release of pip available: 22.3 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import getpass

from rich import print

from langchain.output_parsers import GuardrailsOutputParser

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

OPENAI_API_KEY = getpass.getpass()

In [9]:
input_problem = input("Enter the problem: ")
input_nodes = input("Enter the nodes: ")
input_iterations = input("Enter the iterations: ")

In [15]:
chat = ChatOpenAI(temperature=0.9 , model="gpt-3.5-turbo-16k", client=any , openai_api_key=OPENAI_API_KEY)

formated_problem = {
  "problem": "problem",
  "approach": [
    {
      "approach",
      "score"
    },
    {
      "approach",
      "score"
    },
    {
      "approach",
      "score"
    }
  ]
}


messages = [
    SystemMessage(
        content="You are an AI who is really good at solving problems. You always think step by step and come up with different approaches to solve a problem",
    ),
    HumanMessage(
        content=
        f"""
        Given this problem: {input_problem}
        let solve the problem step by step.
        you have {input_iterations} iterations to solve it.
        come up with {input_nodes} different approaches to solve it.
        
        give each approach a score from 0 to 10 based on how good it is.
        
        give it to me in this format:
        {str(formated_problem)}
        """
    ),
]
step1 = chat(messages).content
print(step1)

{'problem': 'solve world hunger', 'approach': [{'score': 8, 'approach': 'Increase agricultural productivity'}, 
{'score': 7, 'approach': 'Promote sustainable farming practices'}, {'score': 9, 'approach': 'Improve access to 
nutritious food'}]}

In [23]:
def highest_score(solution_input):
  approaches = solution_input['approaches']
  highest_score = max(approaches, key=lambda x: x['score'])
  return highest_score['approach']

In [24]:
extractor_rail = """
<rail version="0.1">

<output>
    <string name="problem" description="problem specified" />
    <list name="approaches" description="List of different approaches to solve the problem">
        <object>
            <string name="approach" description="the approach"/>
            <integer name="score" format="valid-range: 0 10" />
        </object>
    </list>
</output>

<prompt>

given the following problem and steps to solve the problem, please extract the problem and a list of different approaches 

{{approach}}

@complete_json_suffix_v2
</prompt>
</rail>
"""

output_parser = GuardrailsOutputParser.from_rail_string(extractor_rail)

prompt = PromptTemplate(
    template=output_parser.guard.base_prompt,
    input_variables=output_parser.guard.prompt.variable_names,
)

model = OpenAI(temperature=0 , openai_api_key=OPENAI_API_KEY) # type: ignore
solution = model(prompt.format_prompt(approach=step1).to_string())
solution = output_parser.parse(solution)
print(solution)
optimal_solution = highest_score(solution)
print(optimal_solution)

{
    'problem': 'solve world hunger',
    'approaches': [
        {'approach': 'Increase agricultural productivity', 'score': 8},
        {'approach': 'Promote sustainable farming practices', 'score': 7},
        {'approach': 'Improve access to nutritious food', 'score': 9}
    ]
}

Improve access to nutritious food

In [26]:

for i in  range(int(input_iterations)):
    messages = [
        SystemMessage(
            content="You have more iterations to solve the problem. Come up with more approaches",
        ),
        HumanMessage(
            content=
            f"""
            Given this problem: {input_problem}
            let solve the problem step by step.
            you have {(int(input_iterations)-int(i))} iterations to solve it.
            this was your previous approach with the highest score: {optimal_solution} build upon it. and come up with more detailed approaches to solve the problem.
            come up with {input_nodes} different approaches to solve which are the next step the above mentioned highest score approach.
            
            give each approach a score from 0 to 10 based on how good it is.
            
            give it to me in this format:
            {str(formated_problem)}
            """
        ),
    ]
    step = chat(messages).content
    solution = model(prompt.format_prompt(approach=step).to_string())
    solution = output_parser.parse(solution)
    
    print(solution)
    
    optimal_solution = highest_score(solution)
    
    print(f"completed iteration {i+1} out of {input_iterations} and the optimal solution is {optimal_solution}")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000023AEECFDE10>: Failed to resolve 'api.openai.com' ([Errno 11001] getaddrinfo failed)")).
